In [28]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import fdrcorrection

# Chi-Squared Family Counts

In [39]:
family = "TRBJ"

df = pd.read_csv("20230307_single_VJusage_OddsRatio_in_DQB10301DQB10602.csv")

df = df.loc[
    (df["cell_type"] == "CD4") & (df["hla"] == "DQB1*03:01_positive"),
    ["segment", "HLA+CellType+VJ+", "HLA-CellType+VJ+"]
].drop_duplicates()
# df['count'] = wanlu_df['HLA+CellType+VJ+'] + wanlu_df['HLA-CellType+VJ+']
# df['family'] = wanlu_df['segment'].str.slice(0,4)

df = df.rename(columns={'HLA+CellType+VJ+': 'DQB1*0301+', 'HLA-CellType+VJ+': 'DQB1*0301-'})

pattern = r'(TRAV[0-9-]+)DV([0-9]+)'
replacement = r'\1/DV\2'
df['segment'] = df['segment'].replace(pattern, replacement, regex=True)

df_long = pd.melt(df, id_vars='segment', var_name='HLA_type', value_name='count')

family_counts_table = df_long.pivot_table(index='HLA_type', columns='segment', values='count')

family_counts_table = family_counts_table[[x for x in family_counts_table.columns if x.startswith(family)]]

family_counts_table

segment,TRBJ1-1,TRBJ1-2,TRBJ1-3,TRBJ1-4,TRBJ1-5,TRBJ1-6,TRBJ2-1,TRBJ2-2,TRBJ2-3,TRBJ2-4,TRBJ2-5,TRBJ2-6,TRBJ2-7
HLA_type,,,,,,,,,,,,,
DQB1*0301+,10248,9000,2814,3400,6389,4024,12244,6504,10364,1448,8628,1637,13553
DQB1*0301-,35507,31846,10022,12524,21110,12920,45541,23631,37619,5274,30269,5704,48855


In [46]:
df.sum()

segment       TRAV39TRAV13-1TRAV36/DV7TRAV12-3TRAV1-2TRAV13-...
DQB1*0301+                                               361012
DQB1*0301-                                              1283283
dtype: object

In [40]:
# compute usage
DQB0301_pos_total = df['DQB1*0301+'].sum()
DQB0301_neg_total = df['DQB1*0301-'].sum()

usage_df = df.copy()
usage_df["DQB0301_pos_usage"] = df['DQB1*0301+'] / DQB0301_pos_total
usage_df["DQB0301_neg_usage"] = df['DQB1*0301-'] / DQB0301_neg_total
usage_df["DQB0301_pos_to_neg_usage_ratio"] = usage_df["DQB0301_pos_usage"] / usage_df["DQB0301_neg_usage"]
usage_df = usage_df.rename(columns={'segment': 'gene'})
usage_df = usage_df[["gene", "DQB0301_pos_usage", "DQB0301_neg_usage", "DQB0301_pos_to_neg_usage_ratio"]]

In [41]:
family_counts_dict = family_counts_table.to_dict()
family_counts_dict

{'TRBJ1-1': {'DQB1*0301+': 10248, 'DQB1*0301-': 35507},
 'TRBJ1-2': {'DQB1*0301+': 9000, 'DQB1*0301-': 31846},
 'TRBJ1-3': {'DQB1*0301+': 2814, 'DQB1*0301-': 10022},
 'TRBJ1-4': {'DQB1*0301+': 3400, 'DQB1*0301-': 12524},
 'TRBJ1-5': {'DQB1*0301+': 6389, 'DQB1*0301-': 21110},
 'TRBJ1-6': {'DQB1*0301+': 4024, 'DQB1*0301-': 12920},
 'TRBJ2-1': {'DQB1*0301+': 12244, 'DQB1*0301-': 45541},
 'TRBJ2-2': {'DQB1*0301+': 6504, 'DQB1*0301-': 23631},
 'TRBJ2-3': {'DQB1*0301+': 10364, 'DQB1*0301-': 37619},
 'TRBJ2-4': {'DQB1*0301+': 1448, 'DQB1*0301-': 5274},
 'TRBJ2-5': {'DQB1*0301+': 8628, 'DQB1*0301-': 30269},
 'TRBJ2-6': {'DQB1*0301+': 1637, 'DQB1*0301-': 5704},
 'TRBJ2-7': {'DQB1*0301+': 13553, 'DQB1*0301-': 48855}}

In [42]:
chi_square_families_res_dict = {'gene': [], 'chi2':[], 'odds_ratio':[], 'p_value':[]}

pos_total_count, neg_total_count = 0, 0
for gene in family_counts_dict:
    pos_total_count += family_counts_dict[gene]['DQB1*0301+']
    neg_total_count += family_counts_dict[gene]['DQB1*0301-']

for gene in family_counts_dict:
    A = family_counts_dict[gene]['DQB1*0301+']
    B = pos_total_count-A
    C = family_counts_dict[gene]['DQB1*0301-']
    D = neg_total_count-C
    contingency_table = np.array([[A, B],
                                  [C, D]])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    chi_square_families_res_dict['gene'].append(gene)
    chi_square_families_res_dict['chi2'].append(chi2)
    chi_square_families_res_dict['odds_ratio'].append((A*D)/(B*C))
    chi_square_families_res_dict['p_value'].append(p_value)

chi_square_families_res_df = pd.DataFrame(chi_square_families_res_dict)
chi_square_families_res_df['pvalue_fdr'] = fdrcorrection(chi_square_families_res_df['p_value'])[1]

In [43]:
merged_df = usage_df.merge(chi_square_families_res_df, how="inner", on="gene")
merged_df = merged_df.sort_values(by=['pvalue_fdr'])
merged_df.to_csv(f"FINAL_WANLU_{family}_FAMILY_CHISQUARE.csv", sep="\t", index=None)

# Chi-Squared Amino Acid Counts

In [ ]:
TRV = "TRBV" # "TRAV"

df = pd.read_csv("20230307_single_VJusage_OddsRatio_in_DQB10301DQB10602.csv")

df = df.loc[
    (df["cell_type"] == "CD4") & (df["hla"] == "DQB1*03:01_positive"),
    ["segment", "HLA+CellType+VJ+", "HLA-CellType+VJ+"]
].drop_duplicates()
# df['count'] = wanlu_df['HLA+CellType+VJ+'] + wanlu_df['HLA-CellType+VJ+']
# df['TRV'] = wanlu_df['segment'].str.slice(0,4)

df = df.rename(columns={'HLA+CellType+VJ+': 'DQB1*0301+', 'HLA-CellType+VJ+': 'DQB1*0301-'})

pattern = r'(TRAV[0-9-]+)DV([0-9]+)'
replacement = r'\1/DV\2'
df['segment'] = df['segment'].replace(pattern, replacement, regex=True)

df_long = pd.melt(df, id_vars='segment', var_name='HLA_type', value_name='count')

family_counts_table = df_long.pivot_table(index='HLA_type', columns='segment', values='count')

family_counts_table = family_counts_table[[x for x in family_counts_table.columns if x.startswith(TRV)]]

family_counts_table

In [ ]:
## load amino acid sequences for each tcr family
#family_counts_table = pd.read_csv("DGN_family_counts_table.csv").set_index("patid")


AA_seq_df = pd.read_csv("{}_AA_sequences.csv".format(TRV))[['family', 'CDR1_sequence', 'CDR2_sequence']]
AA_seq_df['CDR_seq'] = (AA_seq_df['CDR1_sequence'] + AA_seq_df['CDR2_sequence']).apply(lambda x: list(x))

CDR_NUM_AA = 22 # 22 positions, first 12 are CDR1, then 10 are CDR2
family_sequence_dict = dict()
for index, row in AA_seq_df.iterrows():
    family_sequence_dict[row['family']] = row['CDR_seq']
family_counts_table.iloc[0]

In [ ]:
patid_position_AA_dict = dict()
# We have two conditions, DQB1*0301+ and DQB1*0301-, we will treat as having two patients
NUM_PATIENTS = 2
for i in range(NUM_PATIENTS):
    family_counts = family_counts_table.iloc[i]
    patid = family_counts.name
    position_AA_dict = dict()
    for family, count in family_counts.iteritems(): # this patient's count for each family
        # if we have the AA sequence from IMGT for this family
        # This will only contain either TRAV or TRBV
        if family in family_sequence_dict: 
            if not np.isnan(count): # if counts are not null
                #print(f"adding {family} to counts")
                for pos, AA in enumerate(family_sequence_dict[family]): # for CDR AA at each position
                    if pos not in position_AA_dict:
                        position_AA_dict[pos] = dict()
                    if AA not in position_AA_dict[pos]:
                        position_AA_dict[pos][AA] = 0
                    position_AA_dict[pos][AA] += count
    patid_position_AA_dict[patid] = position_AA_dict
patid_position_AA_dict

In [ ]:
"""
position_AA_dict structure for each patid: 22 positions, different AA counts at each position
{'DQB1*0301+': {0: {'T': 22855,
   'V': 10169,
   'N': 18943,
   'D': 15039,
   'Y': 6944,
   'S': 10388,
   'K': 973,
   'A': 4942},
...}
"""

position_lists = [[] for _ in range(CDR_NUM_AA)]
for patid in patid_position_AA_dict:
    patid_df = pd.DataFrame.from_dict(patid_position_AA_dict[patid]) # get the above dict for each patient
    for position in range(CDR_NUM_AA):
        series = patid_df[position]
        series.name = patid
        # N x M array where N is CDR_NUM_AA and M is NUM_PATIENTS
        position_lists[position].append(series)
# N dataframes each with M rows
position_dfs = [pd.concat(position_list, axis=1, sort=True) for position_list in position_lists]

# for i in range(CDR_NUM_AA):
#     position_dfs[i] = position_dfs[i][position_dfs[i].isnull().sum(axis=1) < NUM_PATIENTS].fillna(0.0).transpose()
#     position_dfs[i].index.name = "HLA_type"
#     #position_dfs[i]["DQB1*0301+"] = [1.0, 0.0]

# for i in range(CDR_NUM_AA):
#     position_dfs[i].to_csv("Wanlu_{}_CDR1_CDR2_position_{}_AA_counts_table.csv".format(TRV, i))

In [ ]:
chi_square_AA_res_dict = {'position': [], 'amino_acid': [], 'chi2': [], 'odds_ratio':[], 'p_value':[]}

## CDR1
TRAV_active_positions = [0, 1, 2, 3, 9, 10, 11]
TRBV_active_positions = [0, 1, 2, 9, 10, 11]

if TRV == "TRAV":
    TRV_active_positions = TRAV_active_positions
else:
    TRV_active_positions = TRBV_active_positions

for i in TRV_active_positions:
    hla_pos_AA_dict = patid_position_AA_dict['DQB1*0301+'][i]
    pos_total_count = sum(hla_pos_AA_dict.values())
    hla_neg_AA_dict = patid_position_AA_dict['DQB1*0301-'][i]
    neg_total_count = sum(hla_neg_AA_dict.values())
    for AA in hla_pos_AA_dict:
        A = hla_pos_AA_dict[AA]
        B = pos_total_count-A
        C = hla_neg_AA_dict[AA]
        D = neg_total_count-C
        contingency_table = np.array([[A, B],
                                      [C, D]])
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        
        chi_square_AA_res_dict['position'].append(i+27)
        chi_square_AA_res_dict['amino_acid'].append(AA)
        chi_square_AA_res_dict['chi2'].append(chi2)
        chi_square_AA_res_dict['odds_ratio'].append((A*D)/(B*C))
        chi_square_AA_res_dict['p_value'].append(p_value)


print("\n")
## CDR2
TRAV_active_positions = [0, 1, 2, 3, 6, 7, 8, 9]
TRBV_active_positions = [0, 1, 2, 3, 7, 8, 9]

if TRV == "TRAV":
    TRV_active_positions = TRAV_active_positions
else:
    TRV_active_positions = TRBV_active_positions

for i in TRV_active_positions:
    # +12 to offset the CDR1 AA's in front 
    hla_pos_AA_dict = patid_position_AA_dict['DQB1*0301+'][i+12]
    pos_total_count = sum(hla_pos_AA_dict.values())
    hla_neg_AA_dict = patid_position_AA_dict['DQB1*0301-'][i+12]
    neg_total_count = sum(hla_neg_AA_dict.values())
    for AA in hla_pos_AA_dict:
        A = hla_pos_AA_dict[AA]
        B = pos_total_count-A
        C = hla_neg_AA_dict[AA]
        D = neg_total_count-C
        contingency_table = np.array([[A, B],
                                      [C, D]])
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        
        chi_square_AA_res_dict['position'].append(i+56)
        chi_square_AA_res_dict['amino_acid'].append(AA)
        chi_square_AA_res_dict['chi2'].append(chi2)
        chi_square_AA_res_dict['odds_ratio'].append((A*D)/(B*C))
        chi_square_AA_res_dict['p_value'].append(p_value)

        
chi_square_AA_res_df = pd.DataFrame(chi_square_AA_res_dict)
chi_square_AA_res_df['pvalue_fdr'] = fdrcorrection(chi_square_AA_res_df['p_value'])[1]
chi_square_AA_res_df[chi_square_AA_res_df["pvalue_fdr"] < 0.05].sort_values(by=['position', 'pvalue_fdr']).to_csv(f"FINAL_WANLU_{TRV}_AMINOACID_CHISQUARE.csv", sep="\t", index=None)

In [ ]:


for position in 
assert(set(hla_pos_AA_dict.keys()) == set(hla_neg_AA_dict.keys()))
for AA in hla_pos_AA_dict:
    contingency_table = np.array([[hla_pos_AA_dict[AA], pos_total_count],
                                  [hla_neg_AA_dict[AA], neg_total_count]])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    chi_square_AA_res_dict['position'].append(position)
    chi_square_AA_res_dict['amino_acid'].append(AA)
    chi_square_AA_res_dict['p_value'].append(p_value)

In [ ]:
len(position_dfs)

In [ ]:
position_dfs[0]

In [ ]:
position_dfs[0]["D"]

In [ ]:
contingency_table = np.array([[20, 30],  # Male
                              [25, 35]]) # Female

# Perform the chi-squared test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-squared statistic:", chi2)
print("p-value:", p_value)
print("Degrees of freedom:", dof)
print("Expected frequencies (based on the null hypothesis):")
print(expected)

In [ ]:
res.summary()

In [ ]:

#print(res.summary())
tcr_df = pd.DataFrame()
tcr_df['coef'] = res.params
tcr_df['se'] = res.bse
tcr_df['pvalue'] = res.pvalues
tcr_df['tvalue'] = res.tvalues
#res_df[y_column] = tcr_df.loc[ind_var_col_name]

In [ ]:
tcr_df

In [ ]:
def do_ols(pat_info_df, usage_table, ind_var, y_cols, cov_cols):
    # ind_var: independent variable for model, e.g. 'DQB1*03:01'
    all_df = pd.merge(pat_info_df, usage_table, right_index=True, left_index=True, how="inner")
#     if binary: # code variable as 0 1
#         ind_var_col_name = "{}_binary".format(ind_var)
#         all_df[ind_var_col_name] = (all_df[ind_var] > 0).astype("float")
#     else: # code variable as 0 1 2
    ind_var_col_name = ind_var
    #all_df[ind_var_col_name] = all_df[ind_var_col_name].astype("float")
    
    #print("doing {} tests. out of {} rows, {} are {} > 0".format(len(y_cols), len(all_df), len(all_df[all_df[ind_var]>0]), ind_var))
    x_columns = [ind_var_col_name] + cov_cols
    res_df = pd.DataFrame()
    for y_column in y_cols:
        X = all_df[x_columns]
        y = all_df[y_column]
        X = sm.add_constant(X) # constant is always added
        mod = sm.OLS(y, X)
        res = mod.fit()
        #print(res.summary())
        tcr_df = pd.DataFrame()
        tcr_df['coef'] = res.params
        tcr_df['se'] = res.bse
        tcr_df['pvalue'] = res.pvalues
        tcr_df['tvalue'] = res.tvalues
        res_df[y_column] = tcr_df.loc[ind_var_col_name]
    res_df = res_df.transpose().reset_index().rename(columns={"index":"family"}).sort_values("pvalue", ascending=True)
    # FDR correction
    res_df['pvalue_fdr'] = fdrcorrection(res_df['pvalue'])[1]
    return res_df